# Numerical Experiments - RTS96 case


## Set up

In [ ]:
# Robert Mieth, 2023
# robert.mieth@rutgers.edu

In [ ]:
import numpy as np
import pandas as pd
from math import ceil
from itertools import product
from scipy.sparse import csr_matrix, csr_array, lil_matrix
import scipy.sparse as scs
import xarray as xr
import math
from functools import reduce

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import torch

# from tqdm.notebook import tqdm, trange
from tqdm import tqdm,trange

import pickle

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.patches import Ellipse
from matplotlib.patches import Rectangle
from matplotlib.patches import Patch
import matplotlib.transforms as transforms
%matplotlib inline

In [ ]:
import ps_data_worker as dw

In [ ]:
DTYPE = torch.double
SEED = 42

# Location of data from https://github.com/GridMod/RTS-GMLC
# expects location of "RTS_Data"
RTS_DATA_DIR = DATA_LOCATION + 'data/RTS_Data'

# other data
NREL_STATIONS_METADATA_FILE = DATA_LOCATION + 'data/nrel_wind_data/wtk_site_metadata.csv'
NREL_STATIONS_TIMEZONE_FILE = DATA_LOCATION + 'data/nrel_wind_data/site_timezone.csv'
NREL_NC_FILES_DIR = DATA_LOCATION + 'data/nrel_wind_data/nc_files/'


In [ ]:
def load_station_data(station, which, return_ds=False, set_timestamps=True):
    station_nc_file = f"{NREL_NC_FILES_DIR}/{which}/{station}.nc"
    station_ds = xr.open_dataset(station_nc_file)
    data_size = station_ds.dims['time']
    start_time = station_ds.attrs['start_time']
    sample_step = station_ds.attrs['sample_period']
    timestamps_raw = np.arange(start_time, start_time+(data_size*sample_step), sample_step)
    timestamps_utc = pd.to_datetime(timestamps_raw, unit='s', utc=True)
    nrel_station_tz_df = pd.read_csv(NREL_STATIONS_TIMEZONE_FILE) 
    if return_ds:
        return station_ds
    else:
        if set_timestamps:
            station_df = station_ds.to_dataframe()
            station_tz = nrel_station_tz_df.iloc[station]['zoneName']
            timestamps_utc = pd.to_datetime(timestamps_raw, unit='s', utc=True)
            timestamps_local = timestamps_utc.tz_convert(station_tz)
            station_df['time_utc'] = timestamps_utc
            station_df['time_local'] = timestamps_local
        return station_df
    
def get_single_period_bus_load_vector(load_dict, act_period):
    area_load = load_dict.query('Year == @act_period[0] and Month == @act_period[1] and Day == @act_period[2] and Period == @act_period[3]')
    area_load = area_load[['1', '2','3']].to_dict(orient='records')[0]
    return np.array([bus['area_load_share']*area_load[str(bus['area'])]/basemva for bus in buses])

def get_single_period_data_vector(df, act_period, entry_list, data_in_pu=False):
    act_data = df.query('Year == @act_period[0] and Month == @act_period[1] and Day == @act_period[2] and Period == @act_period[3]')
    act_data = act_data.to_dict(orient='records')[0]
    if data_in_pu:
        return np.array([act_data[entry['id']] for entry in entry_list])
    else:
        return np.array([act_data[entry['id']]/basemva for entry in entry_list])

def load_station_data(station, which, return_ds=False, set_timestamps=True):
    station_nc_file = f"{NREL_NC_FILES_DIR}/{which}/{station}.nc"
    station_ds = xr.open_dataset(station_nc_file)
    data_size = station_ds.dims['time']
    start_time = station_ds.attrs['start_time']
    sample_step = station_ds.attrs['sample_period']
    timestamps_raw = np.arange(start_time, start_time+(data_size*sample_step), sample_step)
    timestamps_utc = pd.to_datetime(timestamps_raw, unit='s', utc=True)
    nrel_station_tz_df = pd.read_csv(NREL_STATIONS_TIMEZONE_FILE) 
    if return_ds:
        return station_ds
    else:
        if set_timestamps:
            station_df = station_ds.to_dataframe()
            station_tz = nrel_station_tz_df.iloc[station]['zoneName']
            timestamps_utc = pd.to_datetime(timestamps_raw, unit='s', utc=True)
            timestamps_local = timestamps_utc.tz_convert(station_tz)
            station_df['time_utc'] = timestamps_utc
            station_df['time_local'] = timestamps_local
        return station_df

# Prepare PS Data

### Load required data

In [ ]:
# load NREL wind power tool kit metadata
nrel_station_md_df = pd.read_csv(NREL_STATIONS_METADATA_FILE)
nrel_station_tz_df = pd.read_csv(NREL_STATIONS_TIMEZONE_FILE)

In [ ]:
# load rts data
rtsdata = dw.RTSDataSet(RTS_DATA_DIR)
# rtsdata = dw.RTSDataSet(RTS_DATA_DIR, basemva=1.)
psdata = dw.create_ps_data_from_rts_data(rtsdata)

### Prepare system data

In [ ]:
basemva = rtsdata.basemva

buses = psdata.busdata
gens = psdata.gendata
branches = psdata.branchdata

# attach cost info
gencost = dw.create_pwlcost_from_rts_data(rtsdata)
gens = [dict(gen, **{'cost': gencost[i]}) for i,gen in enumerate(gens)]

# separate wind farms
wind_gen_ids = [i for i,g in enumerate(psdata.gendata) if g['fuel']=='Wind']
winds = [gens[i] for i in wind_gen_ids]
gens = [gens[i] for i in range(len(gens)) if i not in wind_gen_ids]
gens_to_remove = wind_gen_ids
# separate other gens with time-series data
rtpv_gen_ids = [i for i,g in enumerate(psdata.gendata) if g['type']=='RTPV']
rtpv_gens = [gens[i] for i in rtpv_gen_ids]
gens_to_remove += rtpv_gen_ids
pv_gen_ids = [i for i,g in enumerate(psdata.gendata) if g['type']=='PV']
pv_gens = [gens[i] for i in pv_gen_ids]
gens_to_remove += pv_gen_ids
hydro_gen_ids = [i for i,g in enumerate(psdata.gendata) if g['type'] in ['HYDRO','ROR']]
hydro_gens = [gens[i] for i in hydro_gen_ids]
gens_to_remove += hydro_gen_ids
# remove synchronous condensers, csp, and storage
gens_to_remove += [i for i,g in enumerate(psdata.gendata) if g['type'] in ['SYNC_COND','CSP','STORAGE']]
# update gens
gens = [gens[i] for i in range(len(gens)) if i not in gens_to_remove]

Nbus = len(buses)
Ngen = len(gens)
Nbranch = len(branches)
Nwind = len(winds)

# basic data vectors
pmax = np.array([gen['pmax_pu'] for gen in gens])
pmin = np.array([0 for gen in gens]) # assume this for now because we are not solving a UC 
smax = np.array([branch['cap_pu'] for branch in branches])

# some maps
def list_to_busmap(thelist):
    locs = [entry['bus'] for entry in thelist]
    entry_list = [i for i in range(len(thelist))]
    return csr_matrix(([1]*len(thelist), (locs, entry_list)), shape=(Nbus, len(thelist)))
gen2bus = list_to_busmap(gens)
wind2bus = list_to_busmap(winds)
pv2bus = list_to_busmap(pv_gens)
rtpv2bus = list_to_busmap(rtpv_gens)
hydro2bus = list_to_busmap(hydro_gens)

# compute ptdf
slack = [i for i,bus in enumerate(buses) if bus['is_slack']]
slack_ind = slack[0]
b_vec = np.array([1/branch['x_pu'] for branch in branches])
b_diag = csr_matrix(np.diag(b_vec))
lines_list = [i for i in range(Nbranch)]*2
fromtobus = [branch['from_bus'] for branch in branches] + [branch['to_bus'] for branch in branches]
data = [1]*Nbranch + [-1]*Nbranch 
inc_mat = csr_matrix((data, (lines_list, fromtobus)))
B_branch = b_diag @ inc_mat
B_bus = inc_mat.T @ B_branch
buses_sans_slack = [i for i in range(Nbus) if i not in slack]
B_bus_sans_slack = B_bus[buses_sans_slack][:,buses_sans_slack]
B_bus_inv_sans_slack = scs.linalg.inv(B_bus_sans_slack)
# add row/col of zeros before index slack
B_bus_pseudoinv = B_bus_inv_sans_slack
for i in slack:
    zero_row = i # insert row before old row of that index
    zero_col = i # insert col before old col of that index
    B_bus_pseudoinv._shape = (B_bus_pseudoinv._shape[0]+1, B_bus_pseudoinv._shape[1]+1)
    B_bus_pseudoinv.indices[B_bus_pseudoinv.indices >= zero_col] += 1
    B_bus_pseudoinv.indptr = np.insert(B_bus_pseudoinv.indptr, zero_row+1, B_bus_pseudoinv.indptr[zero_row])
ptdf = B_branch @ B_bus_pseudoinv
ptdf.data = np.round(ptdf.data, 6)
ptdf.eliminate_zeros()

# for now only use first slope as marginal cost data
cE = np.array([gen['cost'].slopes[0] for gen in gens]) 
cR = cE*2

### Model

In [ ]:
def box_robust_dcopf_problem(mu_init, sigma_init, demand, wind, gamma=0,
            M=1e6, allow_slack=False, quadratic_cost=False, allow_wcurt=False):

    # some additional utility
    A_base = 10
    slack_base = 10
    obj_base = basemva/10
    wcurt_base = 10
    
    # some settings
    FR = 1.0 # reduction of line capacity
    
    # define load as a parameter
    d = cp.Parameter(Nbus, value=demand, name="demand")
    w = cp.Parameter(Nwind, value=wind, name="wind")
    
    # define mean and uncertainty of wind power injections as parameters
    mu = cp.Parameter(Nwind, value=mu_init, name="mu")
    sigma = cp.Parameter(Nwind, value=sigma_init, name="sigma")
        
    # main variables
    p  = cp.Variable(Ngen, pos=True, name="p")
    rp = cp.Variable(Ngen, pos=True, name="rp")
    rm = cp.Variable(Ngen, pos=True, name="rm")
    A  = cp.Variable((Ngen, Nwind), pos=True, name="A")
    fRAMp = cp.Variable(Nbranch, pos=True, name="fRAMp")
    fRAMm = cp.Variable(Nbranch, pos=True, name="fRAMm")

    # aux. variables for robust constraints
    z = cp.Variable((2*Nbranch, Nwind), pos=True, name="z")
    
    # aux. variables to ensure feasibility
    if allow_slack:
        slack = cp.Variable(2*Ngen + 2*Nbranch, pos=True, name="slack")
    if allow_wcurt:
        wcurt = cp.Variable(Nwind, pos=True, name="wcurt")
    
    # basic det constraints
    if allow_wcurt:
        flow = ptdf @ ((gen2bus @ p) + (wind2bus @ (w - wcurt/wcurt_base)) - d)
        balconst = cp.sum(p) + cp.sum(w) - cp.sum(wcurt/wcurt_base) == cp.sum(d)
    else:
        flow = ptdf @ ((gen2bus @ p) + (wind2bus @ w - d))
        balconst = cp.sum(p) + cp.sum(w) == cp.sum(d)
    consts = [
        balconst,
        p + rp <= pmax,
        p - rm >= pmin, 
        A.T @ np.ones(Ngen) == np.ones(Nwind)*A_base,
         flow + fRAMp == smax * FR,
        -flow + fRAMm == smax * FR,
    ]               
    if allow_wcurt:
        consts.append(wcurt <= w*wcurt_base)

    # box support constraints
    for g in range(Ngen):
        if allow_slack:
            consts.append((mu.T @ (-A[g,:]/A_base)) + (sigma.T @ A[g,:]/A_base) <= rp[g] + slack[g]/slack_base)
        else:
            consts.append((mu.T @ (-A[g,:]/A_base)) + (sigma.T @ A[g,:]/A_base) <= rp[g])
        if allow_slack:
            consts.append((mu.T @ (A[g,:]/A_base)) + (sigma.T @  A[g,:]/A_base) <= rm[g] + slack[g+Ngen]/slack_base)
        else:
            consts.append((mu.T @ (A[g,:]/A_base)) + (sigma.T @  A[g,:]/A_base) <= rm[g])
    for l in range(Nbranch):
        Bl = cp.reshape(ptdf[l,:] @ (wind2bus - (gen2bus @ A/A_base)), Nwind)
        # Bl = (ptdf[l,:] @ (wind2bus - (gen2bus @ A))).T
        if allow_slack:
            consts.append(mu.T @ Bl + (sigma.T @ z[l,:]) <= fRAMp[l] + slack[2*Ngen+l]/slack_base)
        else:
            consts.append(mu.T @ Bl + (sigma.T @ z[l,:]) <= fRAMp[l])
        consts.append(z[l,:] >= Bl)
        consts.append(z[l,:] >= -Bl)
        if allow_slack:
            consts.append(mu.T @ -Bl + (sigma.T @ z[Nbranch+l,:]) <= fRAMm[l] + slack[2*Ngen+Nbranch+l]/slack_base)   
        else:
            consts.append(mu.T @ -Bl + (sigma.T @ z[Nbranch+l,:]) <= fRAMm[l])
        consts.append(z[Nbranch+l,:] >= -Bl)
        consts.append(z[Nbranch+l,:] >= Bl)

    # objective
    cost_E = (cE.T @ p) * obj_base
    if quadratic_cost:
        cost_E_quad = cp.sum_squares(cp.multiply(cE_quad, p*obj_base))
    else:
        cost_E_quad = 0                         
    (cE.T @ p)
    cost_R = (cR.T @ (rp + rm)) * obj_base
    objective = cost_E + cost_E_quad + cost_R
    
    thevars = [p, rp, rm, A, fRAMp, fRAMm, z]
    if allow_slack:
        thevars.append(slack)
        penalty_slack = cp.sum(slack) * 1e3
        # penalty_slack = cp.sum_squares(slack)
        objective += penalty_slack
    if allow_wcurt:
        thevars.append(wcurt)
        curt_penalty = cp.sum(wcurt) * 1e3/wcurt_base
        objective += curt_penalty
    
    x = cp.hstack([v.flatten() for v in thevars])
    # regularization = reg_gam*cp.sum_squares(x)
    regularization = gamma*cp.sum_squares(A.flatten())
    objective += regularization
    
    theprob = cp.Problem(cp.Minimize(objective), consts)

    theparams = [d, w, mu, sigma]
    
    return theprob, thevars, theparams, consts

### Test model

In [ ]:
# get a load and wind sample and test feasibility

act_period = (2020,1,1,1)
load_vec = get_single_period_bus_load_vector(rtsdata.timeseries['load_da_regional'], act_period)
wind_vec = get_single_period_data_vector(rtsdata.timeseries['wind_da'], act_period, winds)
pv_vec = get_single_period_data_vector(rtsdata.timeseries['pv_da'], act_period, pv_gens)
rtpv_vec = get_single_period_data_vector(rtsdata.timeseries['rtpv_da'], act_period, rtpv_gens)
hydro_vec = get_single_period_data_vector(rtsdata.timeseries['hydro_da'], act_period, hydro_gens)

net_demand = load_vec - pv2bus@pv_vec - rtpv2bus@rtpv_vec - hydro2bus@hydro_vec

print(f'Period:           {act_period}')
print(f'Total net demand: {sum(net_demand):.2f} p.u.')
print(f'Total wind:       {sum(wind_vec):.2f} p.u.')

mu_init = np.zeros(Nwind)
sigma_init = np.zeros(Nwind)

prob, _, theparams, _ = box_robust_dcopf_problem(mu_init, sigma_init, net_demand, wind_vec, gamma=0, allow_slack=True, allow_wcurt=True)
prob.solve(solver='ECOS')
print(prob.status)
print(f'Objective value:  {prob.value:.4f}')
print()

act_period = (2020,8,1,18)
load_vec = get_single_period_bus_load_vector(rtsdata.timeseries['load_da_regional'], act_period)
wind_vec = get_single_period_data_vector(rtsdata.timeseries['wind_da'], act_period, winds)
pv_vec = get_single_period_data_vector(rtsdata.timeseries['pv_da'], act_period, pv_gens)
rtpv_vec = get_single_period_data_vector(rtsdata.timeseries['rtpv_da'], act_period, rtpv_gens)
hydro_vec = get_single_period_data_vector(rtsdata.timeseries['hydro_da'], act_period, hydro_gens)
net_demand = load_vec - pv2bus@pv_vec - rtpv2bus@rtpv_vec - hydro2bus@hydro_vec

print(f'Period:           {act_period}')
print(f'Total net demand: {sum(net_demand):.2f} p.u.')
print(f'Total wind:       {sum(wind_vec):.2f} p.u.')

theparams[0].value = net_demand
theparams[1].value = wind_vec
prob.solve(solver='ECOS', warm_start=True)
print(prob.status)
print(f'Objective value:  {prob.value:.4f}')

## Data preparation

### NREL wind data

Original data source: https://data.openei.org/s3_viewer?bucket=nrel-pds-wtk&prefix=wtk-techno-economic%2Fpywtk-data%2F

In [ ]:
# load NREL wind power tool kit metadata
nrel_station_md_df = pd.read_csv(NREL_STATIONS_METADATA_FILE)
nrel_station_tz_df = pd.read_csv(NREL_STATIONS_TIMEZONE_FILE)

# get the k=1 closest stations from the NREL wind data set
wind_gen_latlon = [(psdata.busdata[gen['bus']]['lon'], psdata.busdata[gen['bus']]['lat']) for gen in winds]
k = 1
closest_stations = []
distances = []
for w_loc in tqdm(wind_gen_latlon):
    dists = np.array(nrel_station_md_df.apply(lambda station: np.linalg.norm(np.array((station['longitude'], station['latitude'])) - w_loc), axis=1))
    closest_stations.append(np.argsort(dists)[:k])
    distances.append(np.sort(dists)[:k])

if k==1:
    closest_stations = [c[0] for c in closest_stations]
    print(closest_stations)
else:
     pass
    # not yet implemented

In [ ]:
# create a new day ahead and real time wind for the wind generators from the NREL data set
# 2012 seems to be the "most similar" in the NREL data set compared to the RTS data set
# 5% wind scaling for better fit in the winter months
WIND_SCALING = 1.05
WINDYEAR = 2012

wind_i = 0
wind_gen = winds[0]
nrel_station = closest_stations[wind_i]

# get and prepare da data
def prepare_da_wind_data_timesteps(ts):
    return ts.year, ts.month, ts.day, ts.hour+1

nrel_wind_data_da = pd.DataFrame()
metadata = pd.DataFrame()
for wind_i, wind_gen in enumerate(tqdm(winds)):
    nrel_station = closest_stations[wind_i]
    station_df_fc = load_station_data(nrel_station, 'forecasts')
    station_cap = nrel_station_md_df.iloc[nrel_station]['capacity']
    station_df_fc['day_ahead_power_norm'] = station_df_fc['day_ahead_power'].apply(lambda row: row/station_cap)
    
    if wind_i == 0:
        nrel_wind_data_da[['Year','Month','Day','Period']] = station_df_fc.apply(lambda x: prepare_da_wind_data_timesteps(x['time_utc']), axis=1, result_type='expand')
        metadata[['Year','Month','Day','Period']] = station_df_fc.apply(lambda x: prepare_da_wind_data_timesteps(x['time_utc']), axis=1, result_type='expand')
    nrel_wind_data_da[wind_gen['id']] = station_df_fc['day_ahead_power_norm'] * wind_gen['pmax_pu'] * WIND_SCALING

nrel_wind_data_da = nrel_wind_data_da[nrel_wind_data_da['Year'] == WINDYEAR].reset_index(drop=True)
metadata = metadata[metadata['Year'] == WINDYEAR].reset_index(drop=True)

In [ ]:
# inspect data frame
nrel_wind_data_da

### Forecast errors

In [ ]:
station_numbers = closest_stations # legacy TODO
station_error_collection = []

wind_error_collection_df = pd.DataFrame()
rts_wind_ids = [psdata.gendata[wind_gen_ids[si]] for si in range(len(station_numbers))]

for si,station in enumerate(tqdm(station_numbers)):
    rts_wind_gen = psdata.gendata[wind_gen_ids[si]]
    rts_wind_id = rts_wind_gen['id']
    
    # load data
    station_df_fc = load_station_data(station, 'forecasts')
    station_df_rt = load_station_data(station, 'real_time')
    if si == 0:
        wind_error_collection_df['time_utc'] =  station_df_rt['time_utc']
        wind_error_collection_df['total_da_fc'] = 0
        
    # prepare data
    station_cap = nrel_station_md_df.iloc[station]['capacity']
    station_df_fc['day_ahead_power_norm'] = station_df_fc['day_ahead_power'].apply(lambda row: row/station_cap)
    station_df_fc['hour_ahead_power_norm'] = station_df_fc['hour_ahead_power'].apply(lambda row: row/station_cap)
    station_df_rt['power_norm'] = station_df_rt['power'].apply(lambda row: row/station_cap)
    # north wind (i.e., wind FROM the north) is 360°, south wind is 180°
    # mathematical wind direction has west wind aligned with the x-axis, i.e., zero vector angle for west wind (270°)
    station_df_rt['wind_direction_ang'] = station_df_rt['wind_direction'].apply(lambda wwd: math.sin(270-wwd)) 
    # compute forecast errors for day_ahead
    # expand the forecast to 5 mins
    day_ahead_5min = np.repeat(station_df_fc['day_ahead_power_norm'].to_numpy(), 12)
    rt_5min = station_df_rt['power_norm'].to_numpy()
    station_df_rt['error_day_ahead'] = rt_5min - day_ahead_5min
    # compute forecast errors for hour_ahead
    # expand the forecast to 5 mins
    hour_ahead_5min = np.repeat(station_df_fc['hour_ahead_power_norm'].to_numpy(), 12)
    rt_5min = station_df_rt['power_norm'].to_numpy()
    station_df_rt['error_hour_ahead'] = rt_5min - hour_ahead_5min

    # re-construct forecast value
    station_df_rt['da_forecast'] = station_df_rt['power_norm'] - station_df_rt['error_day_ahead']
    
    # aggregate
    wind_error_collection_df['total_da_fc'] += station_df_rt['da_forecast'] * rts_wind_gen['pmax_pu']
    wind_error_collection_df[rts_wind_id] = station_df_rt['error_day_ahead'] * rts_wind_gen['pmax_pu']

In [ ]:
DTYPE = torch.float32
TEST_PERC = 0.25

print(">> preparing time stamps")
wind_error_collection_df[['Year','Month','Day','Period']] = wind_error_collection_df.apply(lambda x: prepare_da_wind_data_timesteps(x['time_utc']), axis=1, result_type='expand')

print(">> preparing data")
all_errors = wind_error_collection_df[['309_WIND_1', '317_WIND_1', '303_WIND_1', '122_WIND_1']].to_numpy()
all_periods = wind_error_collection_df[['Year','Month','Day','Period']].to_numpy()

train_ids, test_ids = train_test_split(np.arange(len(wind_error_collection_df)), test_size=int(len(wind_error_collection_df)*TEST_PERC), random_state=SEED)

# forecast error data 
train = all_errors[train_ids]
test = all_errors[test_ids]

print(f'Created {len(train)} training samples an {len(test)} test samples')

train_data = torch.tensor(train, dtype=DTYPE)
test_data = torch.tensor(test, dtype=DTYPE)

# periods for selection later
train_scenarios = all_periods[train_ids]
test_scenarios = all_periods[test_ids]

# init based on stdv
mu_init = np.mean(train, axis=0)
sigma_init =np.std(train, axis=0)/2

# init based on percentile
perc= 10 # in percent
percupper = np.percentile(train, 100-perc, axis=0)
perclower = np.percentile(train, perc, axis=0)
mu_base_perc = (percupper + perclower) / 2
sigma_base_perc = mu_init + ((percupper - perclower) / 2)

## Cost-based loss

### Loss function

In [ ]:
# use relu to discard negative values
nonneg = torch.nn.ReLU(inplace=False)

def expected_cost(var_values, hist_data_tch, gamma=0):
    
    # some settings
    A_base = 10
    slack_base = 10
    obj_base = basemva/10
    wcurt_base = 10
    
    p = var_values[varname_to_varid_dict['p']]
    rp = var_values[varname_to_varid_dict['rp']]
    rm = var_values[varname_to_varid_dict['rm']]
    A = var_values[varname_to_varid_dict['A']]
    fRAMp = var_values[varname_to_varid_dict['fRAMp']]
    fRAMm = var_values[varname_to_varid_dict['fRAMm']]
    if 'slack' in varname_to_varid_dict.keys():
        slack = var_values[varname_to_varid_dict['slack']]
    else:
        slack = torch.tensor(0)
    if 'wcurt' in varname_to_varid_dict.keys():
        wcurt = var_values[varname_to_varid_dict['wcurt']]
    else:
        wcurt = torch.tensor(0)

    varlist = [p, rp, rm, A, fRAMp, fRAMm, wcurt, slack]
    x = torch.hstack([v.flatten() for v in varlist])
        
    # expected first stage cost
    opf_cost = (torch.dot(p, cE_tch) + torch.dot(rp + rm, cR_tch)) 
    opf_cost += torch.sum(wcurt) * 1e3/wcurt_base 
    opf_cost += torch.sum(slack) * obj_base * 1e3

    # expected reserve violation cost
    reaction_gen = torch.matmul(A/A_base, hist_data_tch.T)
    expected_rp_viol_cost = torch.sum(nonneg(-reaction_gen.T - rp[None, :]).mean(axis=0) * cM_tch)
    expected_rm_viol_cost = torch.sum(nonneg(reaction_gen.T - rm[None, :]).mean(axis=0) * cM_tch)
    reaction_branch = torch.matmul(torch.matmul(ptdf_tch,(wind2bus_tch - torch.matmul(gen2bus_tch, A/A_base))), hist_data_tch.T)
    expected_framp_viol_cost = torch.sum(nonneg(reaction_branch.T - fRAMp[None, :]).mean(axis=0) * cM_tch)
    expected_framm_viol_cost = torch.sum(nonneg(-reaction_branch.T - fRAMm[None, :]).mean(axis=0) * cM_tch)
    
    regularization = gamma*torch.sum(torch.square(x))
    
    return opf_cost + expected_rp_viol_cost + expected_rm_viol_cost + expected_framp_viol_cost + expected_framm_viol_cost + regularization

### P-All

In [ ]:
experiment_label = 'with_prescription_lr-5_gamm0_cM2000'

# some settings
MAX_EPOCH = 100
BATCHSIZE = 10
LR = 1e-6
LMOM = 0.3
GAMMA = 0.

# reset randomness
np.random.seed(seed=SEED)

# constraint violation penalty
cM = 10000

# other parameters
cE_tch, cR_tch, cM_tch = torch.tensor(cE, dtype=DTYPE), torch.tensor(cR, dtype=DTYPE), torch.tensor(cM, dtype=DTYPE)
ptdf_tch = torch.tensor(ptdf.toarray(), dtype=DTYPE)
gen2bus_tch = torch.tensor(gen2bus.toarray(), dtype=DTYPE)
wind2bus_tch = torch.tensor(wind2bus.toarray(), dtype=DTYPE)
zero_tch = torch.tensor(0, dtype=DTYPE)

# get an intial scenario
act_period = (2020,1,1,1)
load_vec = get_single_period_bus_load_vector(rtsdata.timeseries['load_da_regional'], act_period)
wind_vec = get_single_period_data_vector(rtsdata.timeseries['wind_da'], act_period, winds)
pv_vec = get_single_period_data_vector(rtsdata.timeseries['pv_da'], act_period, pv_gens)
rtpv_vec = get_single_period_data_vector(rtsdata.timeseries['rtpv_da'], act_period, rtpv_gens)
hydro_vec = get_single_period_data_vector(rtsdata.timeseries['hydro_da'], act_period, hydro_gens)
net_demand = load_vec - pv2bus@pv_vec - rtpv2bus@rtpv_vec - hydro2bus@hydro_vec

# set up the layer
inner, vs, params, consts = box_robust_dcopf_problem(mu_init, sigma_init, net_demand, wind_vec, gamma=GAMMA, allow_slack=True, allow_wcurt=True, quadratic_cost=False)
inner_cvxpylayer = CvxpyLayer(inner, parameters=inner.parameters(), variables=inner.variables())
varname_to_varid_dict = {key: i for i,key in enumerate(inner.var_dict.keys())}

# set up the prescriptor
sigma_prescriptor = torch.nn.Linear(Nbus+Nwind, Nwind)
sigma_prescriptor.weight.data = torch.zeros((Nwind, Nbus+Nwind))
sigma_prescriptor.bias.data = torch.tensor(sigma_init, dtype=DTYPE)
mu_prescriptor = torch.nn.Linear(Nbus+Nwind, Nwind)
mu_prescriptor.weight.data = torch.zeros((Nwind, Nbus+Nwind))
mu_prescriptor.bias.data = torch.tensor(mu_init, dtype=DTYPE)

# set up SGD 
parameters = list(mu_prescriptor.parameters()) + list(sigma_prescriptor.parameters())
opt = torch.optim.SGD(parameters, lr=LR, momentum=LMOM) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=5)

#train
loss_during_training = []
training_data_df = pd.DataFrame()
# for epoch in trange(MAX_EPOCH, desc="Training progress", unit="epoch"):
with trange(MAX_EPOCH, position=0) as ep_looper:
    for epoch in ep_looper:
        ep_looper.set_description(f'Epoch {epoch}')
        
        # reset loss
        loss = torch.tensor(0., dtype=DTYPE)
        oosloss = torch.tensor(0., dtype=DTYPE)
        baseloss = torch.tensor(0., dtype=DTYPE)
    
        with trange(BATCHSIZE, position=1, leave=False) as batch_looper:
            for batch in batch_looper:
                # create net demand scenario 
                rts_year = 2020
                wind_year = 2012
                scen_mo = np.random.randint(1,12)
                scen_day = np.random.randint(1,28)
                scen_per = np.random.randint(1,24)
                act_period = (rts_year, scen_mo, scen_day, scen_per)
                act_wind = (wind_year, scen_mo, scen_day, scen_per)
                load_vec = get_single_period_bus_load_vector(rtsdata.timeseries['load_da_regional'], act_period)
                wind_vec = get_single_period_data_vector(nrel_wind_data_da, act_wind, winds)
                pv_vec = get_single_period_data_vector(rtsdata.timeseries['pv_da'], act_period, pv_gens)
                rtpv_vec = get_single_period_data_vector(rtsdata.timeseries['rtpv_da'], act_period, rtpv_gens)
                hydro_vec = get_single_period_data_vector(rtsdata.timeseries['hydro_da'], act_period, hydro_gens)
                net_demand = load_vec - pv2bus@pv_vec - rtpv2bus@rtpv_vec - hydro2bus@hydro_vec
                d_scenario = torch.tensor(net_demand, dtype=DTYPE)
                w_scenario = torch.tensor(wind_vec, dtype=DTYPE)
                scenario_vector = torch.cat((d_scenario, w_scenario))

                d_scenario = torch.tensor(net_demand, dtype=DTYPE)
                w_scenario = torch.tensor(wind_vec, dtype=DTYPE)
                scenario_vector = torch.cat((d_scenario, w_scenario))

                # prescribe the the set size
                mu = mu_prescriptor(scenario_vector.float())
                sigma = sigma_prescriptor(scenario_vector.float())

                # compute current inner solution
                opf_params = [w_scenario, d_scenario, mu, sigma]
                var_values = inner_cvxpylayer(*opf_params,  solver_args={'solve_method': "ECOS"})

                # calculate loss
                temploss = expected_cost(var_values, train_data, gamma=GAMMA)
                loss = loss + temploss/BATCHSIZE
              
        # backpropagate
        loss.backward()

        # step the SGD
        opt.step()
        opt.zero_grad()
        scheduler.step(loss)
        
        # some analysis and reporting
        current_results = pd.Series({
            "epoch": epoch,
            "loss": loss.item(),
            "mu_pres_weight": mu_prescriptor.weight.data.detach(),
            "mu_pres_bias": mu_prescriptor.bias.data.detach(),
            "sigma_pres_weight": sigma_prescriptor.weight.data.detach(),
            "sigma_pres_bias": sigma_prescriptor.bias.data.detach(),
        })
        training_data_df = pd.concat([training_data_df, current_results.to_frame().T], ignore_index=True)
        training_data_df.to_csv(f"{experiment_label}.csv")
        loss_during_training.append(loss.item())
        ep_looper.set_postfix(loss=loss.item())

results_with_prescription = training_data_df.copy()
            
# some final reporting    
fig, ax = plt.subplots(1,1)
ax.plot(loss_during_training, label='train')
ax.set_ylabel('loss')
ax.set_xlabel('step')
ax.legend()

### OOS Testing

In [ ]:
GAMMA = 0.

# reset randomness
np.random.seed(seed=2)

cM = 10000
cM_tch = torch.tensor(cM, dtype=DTYPE)

# get final training
def get_prescriptors(training_results):
    final_training_epoch = training_results.iloc[-1]
    sigma_prescriptor = torch.nn.Linear(Nbus+Nwind, Nwind)
    sigma_prescriptor.weight.data = final_training_epoch.sigma_pres_weight
    sigma_prescriptor.bias.data = final_training_epoch.sigma_pres_bias
    mu_prescriptor = torch.nn.Linear(Nbus+Nwind, Nwind)
    mu_prescriptor.weight.data = final_training_epoch.mu_pres_weight
    mu_prescriptor.bias.data = final_training_epoch.mu_pres_bias
    return mu_prescriptor, sigma_prescriptor

# other parameters
cE_tch, cR_tch, cM_tch = torch.tensor(cE, dtype=DTYPE), torch.tensor(cR, dtype=DTYPE), torch.tensor(cM, dtype=DTYPE)
ptdf_tch = torch.tensor(ptdf.toarray(), dtype=DTYPE)
gen2bus_tch = torch.tensor(gen2bus.toarray(), dtype=DTYPE)
wind2bus_tch = torch.tensor(wind2bus.toarray(), dtype=DTYPE)
zero_tch = torch.tensor(0, dtype=DTYPE)

# create model
act_period = (2020,1,1,1)
load_vec = get_single_period_bus_load_vector(rtsdata.timeseries['load_da_regional'], act_period)
wind_vec = get_single_period_data_vector(rtsdata.timeseries['wind_da'], act_period, winds)
pv_vec = get_single_period_data_vector(rtsdata.timeseries['pv_da'], act_period, pv_gens)
rtpv_vec = get_single_period_data_vector(rtsdata.timeseries['rtpv_da'], act_period, rtpv_gens)
hydro_vec = get_single_period_data_vector(rtsdata.timeseries['hydro_da'], act_period, hydro_gens)
net_demand = load_vec - pv2bus@pv_vec - rtpv2bus@rtpv_vec - hydro2bus@hydro_vec
prob, thevars, theparams, _ = box_robust_dcopf_problem(mu_init, sigma_init, net_demand, wind_vec, gamma=GAMMA, allow_slack=True, quadratic_cost=False, allow_wcurt=True)
varname_to_varid_dict = {key: i for i,key in enumerate(prob.var_dict.keys())}

pu_scale = 100
N_OOS = 100
oos_loss_base = []
oos_loss_presc = []
oos_sets_presc = []
oos_errors = []
for oos in trange(N_OOS):

    # create a scenario
    rts_year = 2020
    wind_year = 2012
    scen_mo = np.random.randint(1,12)
    scen_day = np.random.randint(1,28)
    scen_per = np.random.randint(1,24)
    act_period = (rts_year, scen_mo, scen_day, scen_per)
    act_wind = (wind_year, scen_mo, scen_day, scen_per)
    load_vec = get_single_period_bus_load_vector(rtsdata.timeseries['load_da_regional'], act_period)
    wind_vec = get_single_period_data_vector(nrel_wind_data_da, act_wind, winds)
    pv_vec = get_single_period_data_vector(rtsdata.timeseries['pv_da'], act_period, pv_gens)
    rtpv_vec = get_single_period_data_vector(rtsdata.timeseries['rtpv_da'], act_period, rtpv_gens)
    hydro_vec = get_single_period_data_vector(rtsdata.timeseries['hydro_da'], act_period, hydro_gens)
    net_demand = load_vec - pv2bus@pv_vec - rtpv2bus@rtpv_vec - hydro2bus@hydro_vec
    d_scenario = torch.tensor(net_demand, dtype=DTYPE)
    w_scenario = torch.tensor(wind_vec, dtype=DTYPE)
    scenario_vector = torch.cat((d_scenario, w_scenario))
    
    # create a single error occurence
    scenario_id = np.random.randint(0,len(test_data))
    cur_data = test_data[scenario_id]

    ## set base parameters
    theparams[0].value = net_demand
    theparams[1].value = wind_vec
    
    ## base model
    theparams[2].value = mu_base_perc
    theparams[3].value = sigma_base_perc
    prob.solve(solver='ECOS', warm_start=True)
    var_values = [torch.tensor(v.value, dtype=DTYPE) for v in prob.variables()]
    loss_base = expected_cost(var_values, cur_data, gamma=GAMMA)
    oos_loss_base.append(loss_base.item() * pu_scale)
    
    ## prescribed model
    # parametrize prescribed model
    mu_presc, sigma_presc = get_prescriptors(results_with_prescription)
    mu_current = mu_presc(scenario_vector).detach().numpy()
    sigma_current = sigma_presc(scenario_vector).detach().numpy()
    theparams[2].value = mu_current
    theparams[3].value = sigma_current
    prob.solve(solver='ECOS', warm_start=True)
    var_values = [torch.tensor(v.value, dtype=DTYPE) for v in prob.variables()]
    loss_presc = expected_cost(var_values, cur_data, gamma=GAMMA)
    oos_loss_presc.append(loss_presc.item() * pu_scale)
    
    oos_sets_presc.append({"mu": mu_current, "sigma": sigma_current})
    oos_errors.append(cur_data)
 
print(f'Expected cost base: {np.mean(oos_loss_base):.3f}')
print(f'Expected cost P-All: {np.mean(oos_loss_presc):.3f}')